In [1]:
import os
import json
import numpy as np
import lightning.pytorch as pl

import torch
from torch import optim, nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchmetrics import Accuracy 
from torchmetrics.classification import MulticlassF1Score

from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import EarlyStopping

In [3]:
class helicoid_Dataset(Dataset):
    def __init__(self, patient_folders, files, transform=None, mode='labeled'):
        self.data = []
        self.labels = []
        self.transform = transform
        for patient_folder in patient_folders:
            print(f"loading image {patient_folder}")
            patient_folder = os.path.join('/home/martin_ivan/code/own_labels/npj_database/', patient_folder)
            img_data = []
            img_labels = np.load(os.path.join(patient_folder, 'gtMap.npy')).astype(int)
            for file in files:
                img_data_all = np.load(os.path.join(patient_folder, file))
                if mode == 'labeled':
                    # img_data.append(img_data_all[(img_labels !=0) & (img_labels != 4)])
                    img_data.append(img_data_all[(img_labels !=0)])
                elif mode == 'all':
                    img_data.append(img_data_all.reshape(-1, img_data_all.shape[-1]))
                else:
                    raise ValueError("Unknown mode")
            img_data = np.concatenate(img_data, axis=1)

            self.data.append(img_data)
            if mode == 'labeled':
                # self.labels.append(img_labels[(img_labels !=0) & (img_labels != 4)])
                self.labels.append(img_labels[(img_labels !=0)])
            elif mode == 'all':
                self.labels.append(img_labels.reshape(-1))
            else:
                raise ValueError("Unknown mode")
                    
        self.data = np.concatenate(self.data, axis=0)
        self.labels = np.concatenate(self.labels, axis=0)
        print(f"------------- label counts: {np.unique(self.labels, return_counts=True)} -------------")

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx] - 1
        if self.transform:
            x, y = self.transform((x, y))
        return x, y
    
class ToTensor(object):
    def __call__(self, sample):
        x, y = sample
        return torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.long)

class HelicoidDataModule(pl.LightningDataModule):
    def __init__(self, files, batch_size=64, fold="fold1"):
        super().__init__()
        self.batch_size = batch_size
        self.fold = fold
        self.files = files
        self.transform = ToTensor()

        self.setup()

    def setup(self, stage=None):
        with open('/home/martin_ivan/code/own_labels/folds.json') as f:
            folds = json.load(f)

        if stage=="fit":
            self.dataset_train = helicoid_Dataset(folds[self.fold]["train"], self.files, transform=self.transform)
            self.dataset_val = helicoid_Dataset(folds[self.fold]["val"], self.files, transform=self.transform)
        if stage=="test":
            self.dataset_test = helicoid_Dataset(folds[self.fold]["test"], self.files, transform=self.transform)
        if stage=="predict":
            self.dataset_predict = helicoid_Dataset(folds[self.fold]["test"], self.files, transform=self.transform, mode='all')

    def train_dataloader(self):
        return DataLoader(self.dataset_train, batch_size=self.batch_size, shuffle=True, num_workers=16)
    
    def val_dataloader(self):
        return DataLoader(self.dataset_val, batch_size=self.batch_size, shuffle=False, num_workers=16)
    
    def test_dataloader(self): 
        return DataLoader(self.dataset_test, batch_size=self.batch_size, shuffle=False, num_workers=16)
    
    def predict_dataloader(self):
        return DataLoader(self.dataset_predict, batch_size=self.batch_size, shuffle=False, num_workers=16)
    
    def sample_size(self):
        return self.dataset_train.data.shape[1]
    
    def class_distribution(self):
        return np.unique(self.dataset_train.labels, return_counts=True)[1]
    
    def num_classes(self):
        return len(np.unique(self.dataset_train.labels))

In [21]:
class ClassificationModel(pl.LightningModule):
    def __init__(self, input_dim, output_dim, loss_weight, config):
        super().__init__()

        device = "cuda" if torch.cuda.is_available() else "cpu"
        self.loss_weight = Tensor(loss_weight).to(device)
        self.accuracy = Accuracy(task="multiclass", num_classes=output_dim, top_k=1)
        self.f1_score = MulticlassF1Score(num_classes=output_dim, average="macro")

        self.weight_decay = config["weight_decay"]
        self.lr = config["lr"]
        self.num_layers = config["num_layers"]
        self.hidden_dim = config["hidden_dim"]
        self.last_layer_dim = config["last_layer_dim"]

        input_layer = nn.Linear(input_dim, self.hidden_dim)
        hidden_layers = []
        for _ in range(self.num_layers):
            hidden_layers.append(nn.Linear(self.hidden_dim, self.hidden_dim))
            hidden_layers.append(nn.ReLU())
            hidden_layers.append(nn.BatchNorm1d(self.hidden_dim))
        last_layer = nn.Linear(self.hidden_dim, self.last_layer_dim)
        output_layer = nn.Linear(self.last_layer_dim, output_dim)
        self.layers = nn.Sequential(input_layer, nn.ReLU(), nn.BatchNorm1d(self.hidden_dim), *hidden_layers, last_layer, nn.ReLU(), nn.BatchNorm1d(self.last_layer_dim), output_layer)
        # self.layers = nn.Sequential(input_layer, nn.BatchNorm1d(self.hidden_dim), nn.ReLU(), last_layer, nn.BatchNorm1d(self.last_layer_dim), nn.ReLU(), output_layer)

        self.epoch_logits = []
        self.epoch_labels = []
        self.lowest_val_loss = np.inf
        self.best_epoch = 0

    def forward(self, x):
        return self.layers(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y, weight=self.loss_weight)
        accuracy = self.accuracy(logits, y)
        self.log("train/train_loss", loss)
        self.log("train/train_accuracy", accuracy)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        self.eval()
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y, weight=self.loss_weight)
        accuracy = self.accuracy(logits, y)
        self.log("val/val_loss", loss, on_epoch=True, on_step=False)
        self.log("test/test_accuracy", accuracy, on_epoch=True, on_step=False)

        self.epoch_logits.append(logits)
        self.epoch_labels.append(y)
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = nn.functional.cross_entropy(logits, y, weight=self.loss_weight)
        accuracy = self.accuracy(logits, y)
        self.log("test/test_loss", loss, on_epoch=True, on_step=False)
        self.log("test/test_accuracy", accuracy, on_epoch=True, on_step=False)

        self.epoch_logits.append(logits)
        self.epoch_labels.append(y)

    def on_validation_epoch_end(self):
        # calculate f1 score
        epoch_logits = torch.cat(self.epoch_logits, dim=0)
        epoch_labels = torch.cat(self.epoch_labels, dim=0)
        f1_score = self.f1_score(epoch_logits, epoch_labels)
        self.log("val/f1_score", f1_score, sync_dist=True)
        self.epoch_logits.clear()
        self.epoch_labels.clear()

    def on_test_epoch_end(self):
        # calculate f1 score
        epoch_logits = torch.cat(self.epoch_logits, dim=0)
        epoch_labels = torch.cat(self.epoch_labels, dim=0)
        f1_score = self.f1_score(epoch_logits, epoch_labels)
        self.log("test/f1_score", f1_score, sync_dist=True)
        self.epoch_logits.clear()
        self.epoch_labels.clear()

    def on_train_epoch_end(self):
        i = 0
        for layer in self.layers:
            if isinstance(layer, nn.Linear):
                i += 1
                self.logger.experiment.add_histogram(f"layer_{i}/weight", layer.weight, self.current_epoch)
                self.logger.experiment.add_histogram(f"layer_{i}/bias", layer.bias, self.current_epoch)
    
    def configure_optimizers(self):
        optimizer = optim.Adam(self.parameters(), self.lr, weight_decay=self.weight_decay)
        return optimizer

In [5]:
files_hm = ["preprocessed.npy", "heatmaps_osp.npy", "heatmaps_osp_diff.npy", "heatmaps_osp_diff_mc.npy", "heatmaps_icem.npy", "heatmaps_icem_diff.npy", "heatmaps_icem_diff_mc.npy"]
dm_hm = HelicoidDataModule(files=files_hm, fold="fold1")
dm_hm .setup("fit")

files_bl = ["preprocessed.npy"]
dm_bl = HelicoidDataModule(files=files_bl, fold="fold1")
dm_bl.setup("fit")

loading image 005-01
loading image 007-01
loading image 012-01
loading image 012-02
loading image 018-01
loading image 018-02
loading image 019-01
loading image 020-01
loading image 013-01
loading image 015-01
loading image 017-01
loading image 034-02
loading image 035-01
loading image 036-01
loading image 036-02
loading image 038-01
loading image 040-01
loading image 040-02
loading image 043-01
loading image 043-02
loading image 043-04
loading image 050-01
loading image 051-01
loading image 053-01
loading image 055-01
loading image 056-01
loading image 056-02
loading image 057-01
loading image 058-02
------------- label counts: (array([1, 2, 3, 4]), array([179536,  29262,  70340, 277811])) -------------
loading image 004-02
loading image 008-01
loading image 008-02
loading image 022-01
loading image 022-02
loading image 039-01
loading image 039-02
loading image 041-01
loading image 041-02
------------- label counts: (array([1, 2, 3, 4]), array([39410,  3459,  9548, 43357])) ----------

In [29]:
def train(config, dm):
    logger = TensorBoardLogger(config["log_dir"], name="my_model")
    model = ClassificationModel(input_dim=dm.sample_size(), output_dim=dm.num_classes(), loss_weight=dm.class_distribution(), config=config)

    early_stop_callback = EarlyStopping(monitor="val/val_loss", min_delta=0.0, patience=3, verbose=False, mode="min")
    # trainer = pl.Trainer(logger=logger, max_epochs=config["num_epochs"], devices=1, callbacks=[early_stop_callback])
    trainer = pl.Trainer(logger=logger, max_epochs=1, devices=1, callbacks=[early_stop_callback])

    trainer.fit(model, dm.train_dataloader(), dm.val_dataloader())

    # log hyperparameters and metrics
    best_val_loss = early_stop_callback.best_score
    params = {
        "hidden_dim": config["hidden_dim"],
        "num_layers": config["num_layers"],
        "last_layer_dim": config["last_layer_dim"],
        "lr": config["lr"],
        "weight_decay": config["weight_decay"],
    }
    metrics = {"best_val_loss": best_val_loss}
    logger.log_hyperparams(params, metrics)

    return model

In [31]:
np.random.seed(42)
for i in range(10):
    hidden_dim = np.random.randint(4, 40)
    num_layers = np.random.randint(0, 5)
    last_layer_dim = np.random.randint(4, hidden_dim+1)
    exp = np.random.uniform(-5, -2)
    weight_decay = 10**exp
    exp = np.random.uniform(-4, -3)
    lr = 10**exp

    config = {
        "hidden_dim": hidden_dim,
        "num_layers": num_layers,
        "last_layer_dim": last_layer_dim,
        "lr": lr,
        "weight_decay": weight_decay,
        "num_epochs": 50,
    }

    print(config)

    config_bl = config.copy()
    config_bl["log_dir"] = "./logs/bl/"
    model_bl = train(config_bl, dm_bl)

    config_hm = config.copy()
    config_hm["log_dir"] = "./logs/hm/"
    model_hm = train(config_hm, dm_hm)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: ./logs/bl/my_model
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name     | Type               | Params
------------------------------------------------
0 | accuracy | MulticlassAccuracy | 0     
1 | f1_score | MulticlassF1Score  | 0     
2 | layers   | Sequential         | 3.2 K 
------------------------------------------------
3.2 K     Trainable params
0         Non-trainable params
3.2 K     Total params
0.013     Total estimated model params size (MB)


{'hidden_dim': 8, 'num_layers': 0, 'last_layer_dim': 5, 'lr': 0.00034695163352925664, 'weight_decay': 2.9022967000278148e-05, 'num_epochs': 50}


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=1` reached.


ValueError: low >= high